In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['let-s-make-tfrecord-simple-version', 'iwildcam-2019-fgvc6']


In [2]:
%%time

from __future__ import absolute_import, division, print_function

import tensorflow as tf
import IPython.display as display

tf.enable_eager_execution()
print(tf.VERSION)

AUTOTUNE = tf.data.experimental.AUTOTUNE

1.13.1
CPU times: user 780 ms, sys: 104 ms, total: 884 ms
Wall time: 922 ms


In [3]:
%%time

img_size = 10

df = pd.read_csv('../input/iwildcam-2019-fgvc6/train.csv')

f = df['file_name']
id = df['category_id']

all_image_paths = ['../input/iwildcam-2019-fgvc6/train_images/' + fname for fname in f]
all_image_labels = [i for i in id]

paths_labels = dict(zip(all_image_paths[0:img_size], all_image_labels[0:img_size]))

CPU times: user 644 ms, sys: 88 ms, total: 732 ms
Wall time: 760 ms


In [4]:
%%time

mobile_net = tf.keras.applications.DenseNet121(weights='imagenet', input_shape=(192, 192, 3), include_top=False)
mobile_net.trainable=False

Instructions for updating:
Colocations handled automatically by placer.
29089792/29084464 [==============================] - 0s 0us/step
CPU times: user 8.1 s, sys: 988 ms, total: 9.09 s
Wall time: 10.6 s


In [5]:
%%time

ds = tf.data.TFRecordDataset('../input/let-s-make-tfrecord-simple-version/images.tfrec')

def parse(x):
  result = tf.io.parse_tensor(x, out_type=tf.string)
  result = tf.image.decode_jpeg(result, channels=3)
  result = tf.dtypes.cast(result, tf.float32)
#  result = 2 * (result/255.) - 1
  result = result/255.
#  result = tf.reshape(result, [28, 28, 3])
  return result

ds = ds.map(parse, num_parallel_calls=AUTOTUNE)
#ds = ds.map(change_range)


CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 84.8 ms


In [6]:
lables = tf.data.Dataset.from_tensor_slices(tf.cast(all_image_labels, tf.int64))

ds = tf.data.Dataset.zip((ds, lables))

In [7]:
ds = ds.repeat()
ds = ds.batch(32)

In [8]:
model = tf.keras.Sequential([
  mobile_net,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(1024, activation='relu'),
  tf.keras.layers.Dense(23, activation='softmax')])

In [9]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), 
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

In [10]:
#%%time

#model.fit(ds, epochs=10, steps_per_epoch=5000)

In [11]:
model.fit(ds, epochs=5, steps_per_epoch=60)

Epoch 1/5
60/60 [==============================] - 13s 217ms/step - loss: 1.2148 - acc: 0.6932
Epoch 2/5
60/60 [==============================] - 8s 128ms/step - loss: 0.9123 - acc: 0.7255
Epoch 3/5
60/60 [==============================] - 7s 120ms/step - loss: 0.8140 - acc: 0.7531
Epoch 4/5
60/60 [==============================] - 7s 118ms/step - loss: 0.8133 - acc: 0.7516
Epoch 5/5
60/60 [==============================] - 7s 118ms/step - loss: 0.7632 - acc: 0.7589


In [12]:
!ls -al


total 16
drwxr-xr-x 3 root root 4096 May 11 02:37 .
drwxr-xr-x 6 root root 4096 May 11 02:37 ..
drwxr-xr-x 2 root root 4096 May 11 02:37 .ipynb_checkpoints
-rw-r--r-- 1 root root  199 May 11 02:37 __notebook_source__.ipynb


In [13]:
checkpoint_path = "cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)


In [14]:
# include the epoch in the file name. (uses `str.format`)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    period=1)

model.save_weights(checkpoint_path.format(epoch=0))
model.fit(ds, epochs = 5, steps_per_epoch = 60, callbacks = [cp_callback],
          verbose=1)


Consider using a TensorFlow optimizer from `tf.train`.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
Epoch 1/5
59/60 [============================>.] - ETA: 0s - loss: 0.6178 - acc: 0.8014
Epoch 00001: saving model to cp-0001.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
60/60 [==============================] - 8s 140ms/step - loss: 0.6231 - acc: 0.8005
Epoch 2/5
59/60 [============================>.] - ETA: 0s - loss: 0.6599 - acc: 0.7887
Epoch 00002: saving model to cp-0002.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
60/60 [==============================] - 8s 130ms/step - loss: 0.6594 - acc: 0.7891
Epoch 3/5
59/60 [============================>.] - ETA: 0s - loss: 0.6130 - acc: 0.8072
Epoch 00003: saving model to cp-0003.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
60/60 [==============================] - 8s 126ms/step - loss: 0.6123 - acc: 0.8068
Epo

In [15]:
!ls -al

total 195116
drwxr-xr-x 3 root root     4096 May 11 02:47 .
drwxr-xr-x 6 root root     4096 May 11 02:37 ..
drwxr-xr-x 2 root root     4096 May 11 02:37 .ipynb_checkpoints
-rw-r--r-- 1 root root      199 May 11 02:37 __notebook_source__.ipynb
-rw-r--r-- 1 root root       81 May 11 02:47 checkpoint
-rw-r--r-- 1 root root 33186304 May 11 02:47 cp-0000.ckpt.data-00000-of-00001
-rw-r--r-- 1 root root    62538 May 11 02:47 cp-0000.ckpt.index
-rw-r--r-- 1 root root 33186304 May 11 02:47 cp-0001.ckpt.data-00000-of-00001
-rw-r--r-- 1 root root    62538 May 11 02:47 cp-0001.ckpt.index
-rw-r--r-- 1 root root 33186304 May 11 02:47 cp-0002.ckpt.data-00000-of-00001
-rw-r--r-- 1 root root    62538 May 11 02:47 cp-0002.ckpt.index
-rw-r--r-- 1 root root 33186304 May 11 02:47 cp-0003.ckpt.data-00000-of-00001
-rw-r--r-- 1 root root    62538 May 11 02:47 cp-0003.ckpt.index
-rw-r--r-- 1 root root 33186304 May 11 02:47 cp-0004.ckpt.data-00000-of-00001
-rw-r--r-- 1 root root    62538 May 11 02:47 cp-0004.ck

In [20]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'cp-0005.ckpt'

In [21]:
model.load_weights(latest)

In [ ]:
model.fit(ds, epochs = 5, steps_per_epoch = 60, callbacks = [cp_callback],
          verbose=1)

Epoch 1/5
59/60 [============================>.] - ETA: 0s - loss: 0.4078 - acc: 0.8665
Epoch 00001: saving model to cp-0001.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
60/60 [==============================] - 8s 131ms/step - loss: 0.4110 - acc: 0.8661
Epoch 2/5
59/60 [============================>.] - ETA: 0s - loss: 0.4735 - acc: 0.8432
Epoch 00002: saving model to cp-0002.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
60/60 [==============================] - 8s 126ms/step - loss: 0.4721 - acc: 0.8438
Epoch 3/5
59/60 [============================>.] - ETA: 0s - loss: 0.4344 - acc: 0.8660
Epoch 00003: saving model to cp-0003.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
60/60 [==============================] - 8s 128ms/step - loss: 0.4331 - acc: 0.8667
Epoch 4/5
17/60 [=======>......................] - ETA: 5s - loss: 0.4716 - acc: 0.8548

In [19]:
!ls -al

total 195116
drwxr-xr-x 3 root root     4096 May 11 02:49 .
drwxr-xr-x 6 root root     4096 May 11 02:37 ..
drwxr-xr-x 2 root root     4096 May 11 02:37 .ipynb_checkpoints
-rw-r--r-- 1 root root      199 May 11 02:37 __notebook_source__.ipynb
-rw-r--r-- 1 root root       81 May 11 02:49 checkpoint
-rw-r--r-- 1 root root 33186304 May 11 02:47 cp-0000.ckpt.data-00000-of-00001
-rw-r--r-- 1 root root    62538 May 11 02:47 cp-0000.ckpt.index
-rw-r--r-- 1 root root 33186304 May 11 02:49 cp-0001.ckpt.data-00000-of-00001
-rw-r--r-- 1 root root    62538 May 11 02:49 cp-0001.ckpt.index
-rw-r--r-- 1 root root 33186304 May 11 02:49 cp-0002.ckpt.data-00000-of-00001
-rw-r--r-- 1 root root    62538 May 11 02:49 cp-0002.ckpt.index
-rw-r--r-- 1 root root 33186304 May 11 02:49 cp-0003.ckpt.data-00000-of-00001
-rw-r--r-- 1 root root    62538 May 11 02:49 cp-0003.ckpt.index
-rw-r--r-- 1 root root 33186304 May 11 02:49 cp-0004.ckpt.data-00000-of-00001
-rw-r--r-- 1 root root    62538 May 11 02:49 cp-0004.ck